In [13]:
from diffusers import (DDIMScheduler, PNDMScheduler, EulerDiscreteScheduler, 
                       DPMSolverMultistepScheduler, HeunDiscreteScheduler, 
                       EulerAncestralDiscreteScheduler)
from diffusers import AutoPipelineForText2Image
import ipywidgets as widgets

!jupyter nbextension enable --py widgetsnbextension

usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: kernel kernelspec migrate run script troubleshoot

Jupyter command `jupyter-nbextension` not found.


In [14]:
import torch
from ipywidgets import widgets, interactive, VBox, HBox, Output, Button
from IPython.display import display

def generate_images(prompt, num_images, scheduler, steps):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    pipeline = AutoPipelineForText2Image.from_pretrained("runwayml/stable-diffusion-v1-5").to(device)

    # Load the LoRA weights for the model
    pipeline.load_lora_weights("C:/Users/see/Documents/GitHub/Diffusion-Models-for-floor-plan-drafting/Checkpoints_L1/checkpoint-250", weight_name="pytorch_lora_weights.safetensors")

    # Set scheduler 
    if scheduler == "DDIM":
        pipeline.scheduler = DDIMScheduler.from_pretrained("runwayml/stable-diffusion-v1-5", subfolder="scheduler")
    elif scheduler == "PNDM":
        pipeline.scheduler = PNDMScheduler.from_pretrained("runwayml/stable-diffusion-v1-5", subfolder="scheduler")
    elif scheduler == "EulerDiscrete":
        pipeline.scheduler = EulerDiscreteScheduler.from_pretrained("runwayml/stable-diffusion-v1-5", subfolder="scheduler")
    elif scheduler == "DPMSolverMultistep":
        pipeline.scheduler = DPMSolverMultistepScheduler.from_pretrained("runwayml/stable-diffusion-v1-5", subfolder="scheduler")
    elif scheduler == "HeunDiscrete":
        pipeline.scheduler = HeunDiscreteScheduler.from_pretrained("runwayml/stable-diffusion-v1-5", subfolder="scheduler")
    elif scheduler == "EulerAncestralDiscrete":
        pipeline.scheduler = EulerAncestralDiscreteScheduler.from_pretrained("runwayml/stable-diffusion-v1-5", subfolder="scheduler")

    # Generate images based on the prompt
    images = []
    for _ in range(num_images):
        image = pipeline(prompt, num_inference_steps=steps).images[0]
        images.append(image)
    return images

def display_images(prompt, num_images, steps, scheduler):
    images = generate_images(prompt, num_images, scheduler, steps)
    for idx, img in enumerate(images):
        display(img)
        

# Create widgets
prompt_widget = widgets.Textarea(value='', placeholder='Enter your prompt...', description='Prompt:')
num_images_widget = widgets.IntSlider(value=1, min=1, max=4, step=1, description='Number of Images:')
steps_widget = widgets.IntSlider(value=50, min=1, max=100, step=1, description='Inference Steps:')
scheduler_widget = widgets.Dropdown(options=["DDIM", "PNDM", "EulerDiscrete", "DPMSolverMultistep", "HeunDiscrete", "EulerAncestralDiscrete"], value='DDIM', description='Scheduler:')
generate_button = Button(description="Generate")
output = Output()

# Define the function to be triggered by the button
def on_generate_button_clicked(b):
    with output:
        output.clear_output()
        display_images(prompt_widget.value, num_images_widget.value, steps_widget.value, scheduler_widget.value)

generate_button.on_click(on_generate_button_clicked)

# Display the widgets and button
display(VBox([prompt_widget, num_images_widget, steps_widget, scheduler_widget, generate_button, output]))
